In [2]:
import requests
dbsnp_id = "rs183890475"

headers = {"Content-Type": "application/json"}
url = "https://grch37.rest.ensembl.org/vep/human/id"
url_single = f"{url}/{dbsnp_id}?hgvs=1"
response = requests.get(url_single, headers=headers)
response.json()

[{'most_severe_consequence': 'intron_variant',
  'id': 'rs183890475',
  'seq_region_name': '1',
  'assembly_name': 'GRCh37',
  'allele_string': 'A/C/G',
  'input': 'rs183890475',
  'end': 819405,
  'strand': 1,
  'colocated_variants': [{'strand': 1,
    'start': 819405,
    'allele_string': 'A/C/G',
    'end': 819405,
    'seq_region_name': '1',
    'frequencies': {'C': {'gnomadg_fin': 0,
      'eur': 0,
      'eas': 0,
      'gnomadg_amr': 0.0003921,
      'gnomadg_afr': 0.001516,
      'gnomadg': 0.0004531,
      'gnomadg_mid': 0,
      'gnomadg_nfe': 0,
      'gnomadg_sas': 0,
      'gnomadg_eas': 0,
      'afr': 0.0023,
      'sas': 0,
      'gnomadg_asj': 0,
      'gnomadg_ami': 0,
      'af': 0.0008,
      'gnomadg_remaining': 0,
      'amr': 0.0014}},
    'id': 'rs183890475'}],
  'start': 819405,
  'transcript_consequences': [{'consequence_terms': ['intron_variant'],
    'gene_symbol_source': 'Clone_based_ensembl_gene',
    'impact': 'MODIFIER',
    'hgvsc': 'ENST00000594233.1:c

In [1]:
import anndata as ad
import json

adata = ad.read_h5ad("/Users/joeyrich/Desktop/local/varseek/data/ccle_data_base/vk_count_out_test_April8th/adata_cleaned.h5ad")

In [7]:
adata.var["test_string"] = "abc"
adata.var["test_int32"] = 1
adata.var["test_int32"] = adata.var["test_int32"].astype("int32")
adata.var["test_float32"] = 1.0
adata.var["test_float32"] = adata.var["test_float32"].astype("float32")
adata.var["test_bool"] = True
adata.var["test_bool"] = adata.var["test_bool"].astype("bool")
adata.var["test_category"] = "cat1"
adata.var["test_category"] = adata.var["test_category"].astype("category")


In [ ]:


obs_dtypes = {col: str(dtype) for col, dtype in adata.obs.dtypes.items()}
var_dtypes = {col: str(dtype) for col, dtype in adata.var.dtypes.items()}

with open("adata_dtypes.json", "w") as f:
    json.dump({"obs": obs_dtypes, "var": var_dtypes}, f, indent=2)

In [ ]:
adata.write_h5ad("/Users/joeyrich/Desktop/local/varseek/data/ccle_data_base/vk_count_out_test_April8th/adata_cleaned_with_dtypes_test.h5ad")
adata2 = ad.read_h5ad("/Users/joeyrich/Desktop/local/varseek/data/ccle_data_base/vk_count_out_test_April8th/adata_cleaned_with_dtypes_test.h5ad")

In [13]:
adata2.var.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5329695 entries, ENST00000391429:c.194C>T to ENST00000521381:c.2362_2368del
Data columns (total 17 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   vcrs_id                 object  
 1   vcrs_header             object  
 2   seq_ID                  category
 3   mutation_cdna           category
 4   nucleotide_positions    category
 5   actual_variant          category
 6   start_variant_position  category
 7   end_variant_position    category
 8   variant_source          category
 9   vcrs_count              category
 10  vcrs_detected           category
 11  number_obs              category
 12  test_string             category
 13  test_int32              int32   
 14  test_float32            float32 
 15  test_bool               bool    
 16  test_category           category
dtypes: bool(1), category(12), float32(1), int32(1), object(2)
memory usage: 455.3+ MB


In [14]:
import json
import numpy as np
import pandas as pd
import anndata as ad

with open("adata_dtypes.json") as f:
    dtypes = json.load(f)

# Helper to convert string dtype back to actual type
def str_to_dtype(dtype_str):
    if dtype_str == "category":
        return "category"
    try:
        return np.dtype(dtype_str)
    except TypeError:
        return dtype_str  # fallback, e.g., for 'string[python]'

# Reapply dtypes
adata2.obs = adata2.obs.astype({k: str_to_dtype(v) for k, v in dtypes["obs"].items()})
adata2.var = adata2.var.astype({k: str_to_dtype(v) for k, v in dtypes["var"].items()})


In [18]:
adata2.var = adata2.var.astype({k: str_to_dtype(v) for k, v in dtypes["var"].items()})

In [15]:
adata2.var.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5329695 entries, ENST00000391429:c.194C>T to ENST00000521381:c.2362_2368del
Data columns (total 17 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   vcrs_id                 object  
 1   vcrs_header             object  
 2   seq_ID                  category
 3   mutation_cdna           category
 4   nucleotide_positions    category
 5   actual_variant          category
 6   start_variant_position  category
 7   end_variant_position    category
 8   variant_source          category
 9   vcrs_count              category
 10  vcrs_detected           category
 11  number_obs              category
 12  test_string             object  
 13  test_int32              int32   
 14  test_float32            float32 
 15  test_bool               bool    
 16  test_category           category
dtypes: bool(1), category(11), float32(1), int32(1), object(3)
memory usage: 361.9+ MB
